# テキスト生成: 構築したエンドポイントを使用して推論を実行する方法

このサンプルノートブックでは、CyberAgentLM2 をデプロイし、複数のプロンプトエンジニアリングテクニックを利用し検証します。

## 目次
- [エンドポイントのデプロイ](#エンドポイントのデプロイ): [CyberAgentLM2-7B-Chat (CALM2)](https://huggingface.co/cyberagent/calm2-7b-chat) をエンドポイントにデプロイする
- [推論の準備](#推論の準備): [CyberAgentLM2-7B-Chat (CALM2)](https://huggingface.co/cyberagent/calm2-7b-chat) を試すための import やユーティリティメソッドの実装です
- [Zero-Shot を試す](#zero-shot-を試す): Zero-Shot によって、海賊風に回答させることに挑戦します
- [Few-Shot プロンプトを試す](#few-shot-プロンプトを試す): Few-Shot プロンプトによって、センチメント分析に挑戦します
- [質問応答を試す](#質問応答を試す): Amazon CEO 2022 年の書簡に対して、What, How を問う質問応答に挑戦します
- [要約を試す](#要約を試す): Amazon CEO 2022 年の書簡の要約に挑戦します
- [ChatBot を試す](#chatBot-を試す): 会話の履歴をプロンプトに入れることによって、コンテキストに沿った対話に挑戦します
- [計算を試す](#計算を試す): 足算に挑戦します
- [Agent を試す](#Agent-を試す): Agent を活用して足算の回答精度を高めることに挑戦します
- [発展](#発展): こちらの NoteBook にて学び終えた次の一手について記載しています
- [付録: テキスト生成実行時に渡せるパラメータの説明](#付録-テキスト生成実行時に渡せるパラメータの説明): テキスト生成実行時に渡せるパラメータを説明しています

## エンドポイントのデプロイ

SageMaker JumpStart から CyberAgentLM2 を選択し、エンドポイントをデプロイします。

In [ ]:
%pip install --quiet --upgrade sagemaker

In [ ]:
from ipywidgets import Dropdown
from sagemaker.jumpstart.notebook_utils import list_jumpstart_models


dropdown = Dropdown(
    options=list_jumpstart_models("search_keywords includes Text Generation"),
    value="huggingface-llm-calm2-7b-chat-bf16",
    description="Select a JumpStart text generation model:",
    style={"description_width": "initial"},
    layout={"width": "max-content"},
)
display(dropdown)

In [ ]:
model_id = dropdown.value
model_version = "*"

In [ ]:
from sagemaker.jumpstart.model import JumpStartModel

model = JumpStartModel(model_id=model_id, model_version=model_version)
predictor = model.deploy()

## 推論の準備
[CyberAgentLM2-7B-Chat (CALM2)](https://huggingface.co/cyberagent/calm2-7b-chat) を推論するための準備をしましょう。

In [ ]:
import json
import boto3

[CALM2](https://huggingface.co/cyberagent/calm2-7b-chat) では "USER: xxx\nASSISTANT: yyy" というフォーマットでプロンプトを書くことができます。このフォーマットに合わせるためのユーティリティメソッドを実装しておきます。

In [ ]:
def format_prompt (prompt_pairs, system_add=True):
    prompt = [
        f"{uttr['speaker']}: {uttr['text']}" + ("<|endoftext|>" if uttr['speaker'] == "ASSISTANT" else "")
        for uttr in prompt_pairs
    ]
    prompt = "\n".join(prompt)
    if system_add:
        prompt = (
            prompt
            + "\n"
            + "ASSISTANT: "
        )
    return prompt

推論 endpoint にリクエストして、結果を表示するユーティリティメソッドを作成します。

In [ ]:
newline, bold, unbold = '\n', '\033[1m', '\033[0m'
endpoint_name = predictor.endpoint_name

def query_endpoint(payload, do_print = False):
    client = boto3.client('runtime.sagemaker')
    response = client.invoke_endpoint(EndpointName=endpoint_name, ContentType='application/json', Body=json.dumps(payload).encode('utf-8'))
    model_predictions = json.loads(response['Body'].read())
    generated_text = model_predictions[0]['generated_text']
    if do_print:
        print (
            f"Input Text: {payload['inputs']}{newline}"
            f"Generated Text: {bold}{generated_text}{unbold}{newline}")
    return generated_text

ここまでで準備は整いました。まずは、[CALM2](https://huggingface.co/cyberagent/calm2-7b-chat) に記載のサンプルプロンプトを試してみましょう。

In [ ]:
parameters = {
    "max_new_tokens": 128,
    "return_full_text": False,
    "do_sample": True,
    "top_k": 10,
    "repetition_penalty": 1.1,
    "stop": ["<|endoftext|>", "USER:"]
}

In [ ]:
prompt = [
    {
        "speaker": "USER",
        "text": "Hello, you are an assistant that helps me learn Japanese."
    },
    {
        "speaker": "ASSISTANT",
        "text": "Sure, what can I do for you?"
    },
    {
        "speaker": "USER",
        "text": "VRはなんですか。"
    }
]
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

## Zero-Shot を試す
Zero-Shot (質問と回答の例を直接的にプロンプトに指定しない方法) によって口調を変えられるか試してみましょう。ここでは海賊風な回答ができるようになるか試していきます。まずはシンプルに試してみましょう。

In [ ]:
prompt = [
    {
        "speaker":  "USER",
        "text":  "あなたは海賊です。海賊の口調で答えてください。日本の居酒屋に行ってビールを頼む時どんな頼み方をしますか？"
    }
]
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

結果はどうでしょうか? 海賊味が足りないかもしれませんね。ここで、海賊としての振る舞いを事前にプロンプトに含んだ状態で試してみましょう。

In [ ]:
prompt = [
    {
        "speaker":  "USER",
        "text":  "あなたの職業は何ですか？"
    },
    {
        "speaker":  "ASSISTANT",
        "text":  "私の職業は海賊だ。"
    },
    {
        "speaker":  "USER",
        "text":  "海賊はどんなことをしますか？"
    },
    {
        "speaker":  "ASSISTANT",
        "text":  "海賊は荒くれ者の集まりさ。どんなにひどい嵐の中でも酒を飲んで歌うんだ。"
    },
    {
        "speaker":  "USER",
        "text":  "海賊はどんな話し方をするのですか？"
    },
    {
        "speaker":  "ASSISTANT",
        "text":  "丁寧な言葉は使わないね。"
    },
    {
        "speaker":  "USER",
        "text":  "日本の居酒屋に行ってビールを頼む時どんな頼み方をしますか？"
    },    
]
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

いかがでしょうか? 海賊風な回答が返ってきたでしょうか？ セルを実行する度に回答が変化するため何度か実行して結果を観察してみましょう。

## Few-Shot プロンプトを試す
先ほどの例は回答例を示さずに期待する回答を導く方法でした。ここでは、いくつかの回答例を示すことで期待する結果を導いてみましょう。以下の例は、センチメント分析の例です。文章に対してラベル (ポジティブ / ネガティブ / ニュートラル) を回答するようにプロンプトを構成してみます。
まずは、Few-Shot しない場合を試してみましょう。

In [ ]:
prompt = [
    {
        "speaker": "USER",
        "text": "この新しいミュージックビデオは信じられないほど素晴らしかった。この文章のラベル (ポジティブ / ネガティブ / ニュートラル) を返してください。"
    }
]
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

回答はどうでしょうか？ 何度か試してみるとラベル以外の回答をすること気づくでしょう。続いて、Few-Shot プロンプトを試してみましょう。文章とラベルのペアを事前にプロンプトとして与えておくことで回答方法を指示しています。

In [ ]:
prompt = [
    {
        "speaker": "USER",
        "text": "携帯電話のバッテリーが切れるのは嫌です。"
    },
    {
        "speaker": "ASSISTANT",
        "text": "ネガティブ"
    },
    {
        "speaker": "USER",
        "text": "私の一日は:+1。"
    },
    {
        "speaker": "ASSISTANT",
        "text": "ポジティブ"
    },
    {
        "speaker": "USER",
        "text": "これが記事へのリンクです"
    },
    {
        "speaker": "ASSISTANT",
        "text": "ニュートラル"
    },
    {
        "speaker": "USER",
        "text": "この新しいミュージックビデオは信じられないほど素晴らしかった"
    }
]
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

Few-Shot を使う場合と使わない場合とで複数回実行してみてください。Few-Shot の方が期待する回答が多い傾向に気づくはずです。

## 質問応答を試す
[2022年 Amazon CEO の書簡](https://www.aboutamazon.jp/news/company-news/ceo-andy-jassys-2022-letter-to-shareholders)の第一段落を使用して質問応答を試してみましょう。質問対象となる文章をプロンプトに含めておきます。その文章に対して質問し、正しく回答できるかを観察してみましょう。ここではいくつか質問を試してみたいと思います。簡易的ですがプロンプトを template 化しておきましょう。

In [ ]:
def template_question_answering(question):
    ret = [
        {
            "speaker": "USER",
            "text": f"""\
当社が毎年、発行している株主の皆様への書簡をCEOとして執筆するのは2年目となりますが、
私は今、机に向かいながらAmazonの将来に期待と高揚感を感じています。
マクロ経済において2022年は、ここ数年間に記憶している中でもとりわけ困難な年でした。
当社においてもいくつかの経営上の課題が浮上しましたが、
（過去最高の成長を記録した新型コロナウイルス感染症のパンデミック期の前半からさらに）需要を拡大させることができました。
お客様の体験（カスタマーエクスペリエンス）を短期的、長期的に意味のある形で向上させていくため、
主要事業の変革を推進しました。
そして、お客様、株主の皆様、社員にさらに貢献できる企業となっていくために必要な長期的投資を継続しつつ、
投資判断や今後の発明の在り方や進め方についても重要な調整を行いました。

上記の文章から答えてください。{question}
    """
        }
    ]
    return ret

In [ ]:
question = "CEO が 書簡を書くのは何回目でしょうか?"
prompt = template_question_answering(question)
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

皆さんも自分ならどう回答するかを想定しながら、その回答とモデルが生成した回答とを比較してみてください。複数回実行して回答の変化を観察してみましょう。
次は、先ほどより少し難しい質問をしてみます。

In [ ]:
question = "お客様の体験を向上させるために何をしましたか?"
prompt = template_question_answering(question)
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

このように、プロンプトに回答の参考になる文章を含めることで質問応答することが可能になります。このプロンプトエンジニアリングは [RAG](https://aws.amazon.com/jp/blogs/news/quickly-build-high-accuracy-generative-ai-applications-on-enterprise-data-using-amazon-kendra-langchain-and-large-language-models/) と呼ばれる方式でも重要な考え方の一つです。このサンプルプロンプトにより、[CALM2](https://huggingface.co/cyberagent/calm2-7b-chat) モデルの RAG への有用性を感じることができるでしょう。question に様々な質問を入力して試してみてください。

## 要約を試す
ここでは、要約を試してみましょう。要約は長い文章を短しつつも必要な情報を残すことが重要です。質問応答のテンプレートを再利用して、要約の結果を観察してみましょう。

In [ ]:
question = "上記の文章を要約してください。"
prompt = template_question_answering(question)
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

複数回実行して結果を観察してみましょう。皆さんが期待する要約ができているでしょうか確認してみましょう。

## ChatBot を試す
ここでは、ChatBot としての回答を観察してみましょう。ChatBot に求められる機能として、会話のコンテキスト (文脈) を考慮することが挙げられます。
プロンプトに会話のやり取りを追加しておくことで会話のコンテキストを保ちます。これを実現するための簡易なユーティリティメソッドを用意します。

In [ ]:
class ContextualPrompt:
    def __init__(self):
        self._contextual_prompt = ""
    
    @property
    def contextual_prompt(self):
        return self._contextual_prompt
    
    def add_context(self, prompt, need_nl = False):
        if need_nl:
            self._contextual_prompt += "\n"
        self._contextual_prompt += prompt
    
    def add_speaker_context(self, speaker, text, need_nl = False):
        prompt = [
            {
                "speaker": speaker,
                "text": text
            }
        ]
        self.add_context(format_prompt(prompt), need_nl)

In [ ]:
prompt = [
    {
        "speaker": "USER",
        "text": "私はエンジニアです。あなたはエンジニアアシスタントとして設計にアドバイスしてください。"
    },
    {
        "speaker": "ASSISTANT",
        "text": "承知しました。"
    },
    {
        "speaker": "USER",
        "text": "AWS でシンプルなアプリケーションを作ろうとしています。データベースは何を使うのが良いでしょうか？なるべく運用したくありません。"
    },
    {
        "speaker": "USER",
        "text": "用途によりますが、Amazon RDS (Relational Database Service) for MySQL, PostgreSQL, Oracle, SQL Server, Amazon Auroraを推奨します。これらのデータベースは、高性能でスケーラブルなパフォーマンスを提供し、AWSのグローバルインフラストラクチャを活用して可用性と災害復旧を実現します。また、AWS Management Consoleから簡単に操作できることから、運用コストを大幅に削減できます。"
    },
]
formated_prompt = format_prompt(prompt, False)
chat_prompt = ContextualPrompt()
chat_prompt.add_context(formated_prompt)
print(chat_prompt.contextual_prompt)

まずは、ChatBot のキャラクター設定が反映されているか確認してみましょう。

In [ ]:
prompt = [
    {
        "speaker": "USER",
        "text": "どのような運用が必要ですか？"
    }
]
formated_prompt = format_prompt(prompt, True)
chat_prompt.add_context(formated_prompt, True)
print(chat_prompt.contextual_prompt)

In [ ]:
payload = {
    "inputs": chat_prompt.contextual_prompt,
    "parameters": parameters
}
response_text = query_endpoint(payload)
print(response_text)

過去の文脈を汲んで Amazon RDS の運用について答えていたら成功です。会話履歴を反映したチャットが可能なことが確認できました。
それでは、会話してみましょう。

## 計算を試す
生成系 AI は一般的に計算が苦手と言われています。[CALM2](https://huggingface.co/cyberagent/calm2-7b-chat) はどうか確認してみましょう。どのようなプロンプトを得意とするのか試して観察することによりノウハウとして得ることはビジネスユースにおいて重要です。

In [ ]:
prompt = [
    {
        "speaker": "USER",
        "text": "3 + 5 ="
    }
]
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
payload = {
    "inputs": formated_prompt,
    "parameters": parameters
}
query_endpoint(payload)

結果を確認してみましょう。期待する結果が返ってきたでしょうか? 複数回実行して結果を観察してみましょう。

## Agent を試す
[計算を試す](#計算を試す)では直接足算を LLM に指示しました。しかし、私たちもそうであるように、電卓を使った方が正確に計算できます。この考え方を実現する方法が Agent です。Agent には [ReAct](https://arxiv.org/abs/2210.03629) などさまざまな手法が提案されており、およそ以下のような要素の組合せで実現されます。

- 質問: ユーザ入力
- 考察 (Thought): 質問に対する LLM の出力であり、行動の理由となるもの
- 行動 (Action): 考察に基づき、LLM が採用すべき行動 (例えば 電卓を打つ) 
- 観測 (Observation): 行動から得られた結果であり、完了しない場合に質問に含めて考察、行動を繰り返すことでコンテキストを追加する
- 完了 (Finish): 最終的な結果が得られたか、得られなければ質問、考察、行動を繰り返す

ここでは、簡易化して、質問、考察から行動が導けるのかを足算を例に挑戦してみましょう。

まずは、先ほどと同じ方法で 10 回試して結果を観察しましょう。どれくらい正しく計算できているでしょうか。

In [ ]:
prompt = [
    {
        "speaker": "ユーザー",
        "text": "3 + 5 ="
    }
]
formated_prompt = format_prompt(prompt)
print(formated_prompt)

In [ ]:
for i in range(10):
    payload = {
        "inputs": formated_prompt,
        "parameters": parameters
    }
    print(query_endpoint(payload))
    print("---")

続いて、Agent のアイデアに沿った Few-Shot を利用してみましょう。まずは、Tool となる電卓の準備です。簡易的に足算を表す式を `eval` するだけの実装にしています。

In [ ]:
def dentaku(expression):
    return eval(expression)

Few-Shot を用意します。質問、考察、行動という言葉が持つ意味に引っ張られる傾向がある場合は、それぞれを表すランダムな文字列を利用することで、改善できる場合があります。これも LLM の特性に合わせたプロンプトエンジニアリングの一つです。LLM の特性を理解するために様々なプロンプトを試してみることの重要性が理解できます。ここでは、そのままの単語を利用してみましょう。

In [ ]:
d = {
    "質問": "質問",
    "考察": "考察",
    "行動": "行動",
}

In [ ]:
def dentaku_agent_template(question, arg):
    prompt = [
        {
            "speaker": "USER",
            "text": f"{d['質問']} {question}"
        },
        {
            "speaker": "ASSISTANT",
            "text": f"{d['考察']} {arg} これは数式です。数式は Dentaku を使うべきです。{d['行動']} Dentaku[{arg}]"
        },
    ]
    return format_prompt(prompt, False)

agent_prompt = ContextualPrompt()
agent_prompt.add_context(dentaku_agent_template("1 + 1 =?", "1+1"))
agent_prompt.add_context(dentaku_agent_template("2 x 2 =?", "2*2"), True)
agent_prompt.add_context(dentaku_agent_template("4 / 2 =?", "4/2"), True)
agent_prompt.add_context(dentaku_agent_template("3 - 2 =?", "3-2"), True)

print(agent_prompt.contextual_prompt)

四則演算の Few-Shot ができました。それでは最後に求めたい計算式を入力します。

In [ ]:
question = "3 + 5 =?"
agent_prompt.add_speaker_context("USER", f"{d['質問']} {question}", True)
print(agent_prompt.contextual_prompt)

準備ができました。正しい考察と行動が導けるか確認してみましょう。

In [ ]:
payload = {
    "inputs": agent_prompt.contextual_prompt,
    "parameters": parameters
}
response_text = query_endpoint(payload)
print(response_text)

正しい行動が導けたでしょうか？ 導けたなら、残りは行動に合わせて Tool を実行するハンドラーを実装するだけです。

In [ ]:
import re

def handler(response_tex):
    try:
        target_string = "Dentaku"
        if target_string in response_tex:
            pattern = r'\[(.*?)\]'
            matches = re.findall(pattern, response_tex)
            if matches:
                return dentaku(matches[0])
            else:
                print("Dentaku を使用しますが。計算式がわかりません。")
        else:
            print("適切な Tool がありません。")
    except Exception as e: 
        print(f"エラーが発生しました。判断できません。{e}")

In [ ]:
handler(response_text)

ここまでの処理を複数回実行しやすいように 1 つのセルにまとめて実行してみましょう。LLM に直接足算を解かせる方法に比べて正しい回答が導けるようになったでしょうか? Tool として正しく Dentaku が選択されるでしょうか? また、計算は正しいでしょうか?

In [ ]:
def agent(question):
    # Few-Shot の生成
    agent_prompt = ContextualPrompt()
    agent_prompt.add_context(dentaku_agent_template("1 + 1 =?", "1+1"))
    agent_prompt.add_context(dentaku_agent_template("2 x 2 =?", "2*2"), True)
    agent_prompt.add_context(dentaku_agent_template("4 / 2 =?", "4/2"), True)
    agent_prompt.add_context(dentaku_agent_template("3 - 2 =?", "3-2"), True)

    # 質問の入力
    agent_prompt.add_speaker_context("USER", f"{d['質問']} {question}", True)

    # 考察と行動を導く
    payload = {
        "inputs": agent_prompt.contextual_prompt,
        "parameters": parameters
    }
    response_text = query_endpoint(payload)
    print(f"{response_text}")

    return handler(response_text)

In [ ]:
for i in range(10):
    answer = agent("3 + 5 =?")
    print(f"回答: {answer}")
    print("---")

計算式以外の質問に対しては適切な Tool がないと回答されることが期待されます。期待通り動作する確認してみましょう。

In [ ]:
for i in range(10):
    answer = agent("あなたの名前は?")
    print(f"回答: {answer}")
    print("---")

より高度に Agent を実装するなら"完了"を判断する仕組みを導入し、"完了"するまで質問、考察、行動を自動で繰り返す方法が考えられます。また、考察、行動により得られた結果を質問のプロンプトに含めることでコンテキストを考慮することができます。これらを実装する際には、[LangChain]( https://python.langchain.com/docs/get_started/introduction) の Agents モジュールを利用することができます。

## 発展
- これらのプロンプトは一例です。より多くのユースケースがあります。どのようなユースケースに有用なモデルなのか、様々なプロンプトを与えて観察してみましょう。
- 推論時のパラメータを変えて試してみましょう。生成停止条件やペナルティの与え方、確率によって回答が変わる幅などを変えることで回答がどのように変化するか観察してみましょう。
- 推論 Endpoint の使い方に慣れることができたら、皆さんのシステムに組み込んでみましょう。もし、VPC や 暗号化が必要な場合は deploy 時に設定することができます。是非、挑戦してみましょう。
- [aws-ml-jp](https://github.com/aws-samples/aws-ml-jp) は AWS で機械学習モデルを構築、学習、デプロイする方法が学べる Notebook と教材集です。生成系 AI に関するサンプルもあります。探索してみましょう。

## 付録: テキスト生成実行時に渡せるパラメータの説明
[Hugging Face の Text Generation Inference](https://huggingface.co/blog/sagemaker-huggingface-llm) に則り、以下のパラメータをテキスト生成実行時に渡すことができます。この Notebook では、`max_new_tokens`, `repetition_penalty` などが該当します。

- temperature: モデル内のランダム性を制御します。低い値はモデルをより決定論的にし、高い値はモデルをよりランダムにします。デフォルト値は1.0です。
- max_new_tokens: 生成するトークンの最大数です。デフォルト値は20で、最大値は512です。
- repetition_penalty: 繰り返しの発生確率を制御します。デフォルトはnullです。
- seed: ランダム生成に使用するシードです。デフォルトはnullです。
- stop: 生成を停止するトークンのリストです。これらのトークンのいずれかが生成されると、生成は停止します。
- top_k: 最も高い確率の語彙トークンを保持する数を制御します。デフォルト値はnullで、トップ K フィルタリングを無効にします。
- top_p: デフォルトはnullで、nucleus sampling のための最も高い確率の語彙トークンを保持するための累積確率を制御します。
- do_sample：サンプリングを使用するかどうかです。デフォルト値はfalseです。それ以外の場合、サンプリングなしでデコードされます。
- best_of: 最大 log 確率のトークンで生成した best_of シーケンスを返すかどうかです。デフォルトはnullです。
- details: 生成に関する詳細情報を返すかどうかです。デフォルト値は false です。
- return_full_text: 完全なテキストを返すかどうかです。デフォルト値は false です。それ以外の場合、生成された部分のみを返します。
- truncate: モデルの最大長に切り詰めるかどうかです。デフォルト値は true です。
- typical_p: トークンの典型的な確率です。デフォルトは null です。
- watermark: 生成に使用するウォーターマークです。デフォルト値は false です。